In [1]:
##by ltecho@whu.edu.cn
##
import pandas as pd
import numpy as np
from pyecharts import Pie
from pyecharts import Graph

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "C:\Users\Simpl\Anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [2]:
##读取题目二中的结果文件，ip的终端指纹列表，恶意样本列表，关联社区ip列表等信息
ip_data_cluster = pd.read_csv('ip_file\\ip_data_friends.csv',delimiter=',',encoding='utf-8',dtype='str')

In [3]:
##可视化ip的终端指纹，恶意样本，绑定域名社区关系
mid_nodes = []
mid_links = []
mid_nodes_set = set()
bind_domain_nodes = []
bind_domain_links = []
bind_domain_nodes_set = set()
attack_file_nodes = []
attack_file_links = []
attack_file_nodes_set = set()
for index,ip_data in ip_data_cluster.iterrows():
    #nodes.append({"name":ip_data.ip,"symbolSize":5})
    if str(ip_data.mid_friends_list)!="set()" and str(ip_data.mid_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        mid_friends_list = eval(ip_data.mid_friends_list)
        mid_nodes_set.add(ip_data.ip)
        for k in mid_friends_list:
            mid_nodes_set.add(k)
            mid_links.append({"source":ip_data.ip,"target":k})
    if str(ip_data.bind_domain_friends_list)!="set()" and str(ip_data.bind_domain_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        bind_domain_friends_list = eval(ip_data.bind_domain_friends_list)
        bind_domain_nodes_set.add(ip_data.ip)
        for k in bind_domain_friends_list:
            bind_domain_nodes_set.add(k)
            bind_domain_links.append({"source":ip_data.ip,"target":k})
    if str(ip_data.attack_file_friends_list)!="set()" and str(ip_data.attack_file_friends_list)!='nan':
        #print(ip_data.mid_friends_list)
        attack_file_friends_list = eval(ip_data.attack_file_friends_list)
        attack_file_nodes_set.add(ip_data.ip)
        for k in attack_file_friends_list:
            attack_file_nodes_set.add(k)
            attack_file_links.append({"source":ip_data.ip,"target":k})        
for j in mid_nodes_set:
    mid_nodes.append({"name":j,"symbolSize":5})
for z in bind_domain_nodes_set:
    bind_domain_nodes.append({"name":z,"symbolSize":5})
for z in attack_file_nodes_set:
    attack_file_nodes.append({"name":z,"symbolSize":5})


In [4]:
print('终端指纹ip节点数：'+str(len(mid_nodes)))
print('终端指纹ip关系数：'+str(len(mid_links)))
print('恶意样本ip节点数：'+str(len(attack_file_nodes)))
print('恶意样本ip关系数：'+str(len(attack_file_links)))
print('绑定域名ip节点数：'+str(len(bind_domain_nodes)))
print('绑定域名ip关系数：'+str(len(bind_domain_links)))

终端指纹ip节点数：290
终端指纹ip关系数：1831
恶意样本ip节点数：290
恶意样本ip关系数：4036
绑定域名ip节点数：8
绑定域名ip关系数：4


In [5]:
mid_graph = Graph("终端指纹力导图",width = 1800, height=1400)
mid_graph.add(
    "",
    mid_nodes,
    mid_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [6]:
mid_graph.render('ip_file\\终端指纹力导图1.html')

In [7]:
bind_domain_graph = Graph("绑定域名力导图",width = 1800, height=1400)
bind_domain_graph.add(
    "",
    bind_domain_nodes,
    bind_domain_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [8]:
bind_domain_graph.render('ip_file\\绑定域名力导图1.html')

In [9]:
attack_file_graph = Graph("恶意样本力导图",width = 1800, height=1400)
attack_file_graph.add(
    "",
    attack_file_nodes,
    attack_file_links,
    graph_layout = "force",
    label_pos = "right",
    graph_repulsion = 10,
    line_curve = 0.2,
)

In [10]:
attack_file_graph.render('ip_file\\恶意样本力导图1.html')

In [11]:
#结合三种关联关系分析，进一步得到攻击者和攻击机构的关系网，
###用作具体分析某ip或ip社区的关系


#得到恶意样本控制中心IP列表
attack_file_df = pd.DataFrame(attack_file_links)
attacker_file_df_counts = attack_file_df.groupby('source').count()
attacker_file_df_counts = attacker_file_df_counts.reset_index()
attacker_file_df_counts.rename(columns={'target':'target_number'},inplace=True)

#终端指纹关系列表
mid_df = pd.DataFrame(mid_links)

#得到终端指纹关系中，源ip为恶意样本控制中心的关系数据
mid_attack_file_df = pd.merge(mid_df,attacker_file_df_counts,on='source',how='left')
attacker_file_df_counts['target'] = attacker_file_df_counts['source'].apply(lambda x:x)
mid_attack_file_df.dropna(axis=0,how='any',inplace=True)

#得到终端关系中，目标ip为恶意样本控制中心的关系数据
attack_file_df_target = attacker_file_df_counts[['target','target_number']]
mid_attack_file_df2 = pd.merge(mid_df,attack_file_df_target,on='target',how='left')
mid_attack_file_df2.dropna(axis=0,how='any',inplace=True)

#拼接两份关系数据，得到终端指纹和恶意样本关系的关联数据
mid_attack_file_frames = [mid_attack_file_df,mid_attack_file_df2]
mid_attack_file_data = pd.concat(mid_attack_file_frames,keys=['s','t'])

In [12]:
##可视化查看
##查看恶意样本控制中心ip列表，关联关系最多的列表
attacker_file_df_counts.sort_values(by='source')

,source,target_number,target
0,218.58.75.140,289,218.58.75.140
1,223.100.159.142,289,223.100.159.142
2,36.102.222.104,286,36.102.222.104
3,59.45.61.162,289,59.45.61.162
4,59.46.137.3,289,59.46.137.3
5,59.46.196.254,289,59.46.196.254
6,59.46.196.90,289,59.46.196.90
7,59.46.212.196,289,59.46.212.196
8,59.46.219.83,289,59.46.219.83
9,59.46.227.90,289,59.46.227.90


In [13]:
###查看终端指纹中心点ip：120.200.167.93与恶意样本关系网中控制中心ip列表的关系
mid_attack_file_data.query('source =="120.200.167.93"').sort_values(by='target')

source           target  target_number
t 353  120.200.167.93    218.58.75.140          289.0
  362  120.200.167.93  223.100.159.142          289.0
  490  120.200.167.93   36.102.222.104          286.0
  427  120.200.167.93     59.45.61.162          289.0
  252  120.200.167.93      59.46.137.3          289.0
  241  120.200.167.93    59.46.196.254          289.0
  318  120.200.167.93     59.46.196.90          289.0
  310  120.200.167.93    59.46.212.196          289.0
  304  120.200.167.93     59.46.219.83          289.0
  441  120.200.167.93     59.46.227.90          289.0
  497  120.200.167.93     59.47.37.164          289.0
  438  120.200.167.93     59.47.37.197          288.0
  471  120.200.167.93     59.47.37.198          284.0
  383  120.200.167.93      59.47.37.83          288.0

In [14]:
###针对ip：120.200.167.93，的攻击目标，和攻击能力分析
most_attacker = ip_data_cluster.query('ip=="120.200.167.93"')
##获取社区
most_mid_friends=list(eval(most_attacker.mid_friends_list[2345]))
most_mid_friends.append('120.200.167.93')
most_friends = pd.DataFrame({'ip':most_mid_friends})
#获取社区相关的威胁情报数据
ioc_data = pd.read_csv('ip_file\\ioc_data.csv',delimiter=',',encoding='utf-8')
most_friends_info = pd.merge(most_friends,ip_data_cluster,on='ip',how='left')
most_friends_info = pd.merge(most_friends_info,ioc_data,on='ip',how='left')

In [15]:
#攻击者的恶意样本数
most_friends_info['attack_file_name_number'] = most_friends_info['attack_file_list'].apply(lambda x:len(eval(x)))

In [16]:
#查看社区的恶意标签
most_friends_info.malicious_label.value_counts()

['DDOS', 'WEB_ATTACKER']                       7
[]                                             5
['DDOS', 'SCANNER', 'WEB_ATTACKER']            4
['DDOS']                                       3
['SCANNER', 'WEB_ATTACKER']                    2
['DDOS', 'SPAM', 'WEB_ATTACKER']               1
['SCANNER', 'SPAM']                            1
['WEB_ATTACKER']                               1
['DDOS', 'SCANNER', 'SPAM', 'WEB_ATTACKER']    1
['DDOS', 'SCANNER']                            1
Name: malicious_label, dtype: int64

In [17]:
#攻击者的解析域名数,当前数据只是第二题结果文件的部分数据，
#most_friends_info['resolve_domain_number'] = most_friends_info['resolve_domain_list'].apply(lambda x:len(eval(x)) if type(x)==str else 0)

In [18]:
##从解析域名的角度，分析攻击者的攻击目的，
ip_log_dns_cate = pd.read_csv('ip_file\\ip_log_dns_cate.csv',delimiter=',',encoding='utf-8')
most_dns_cate = ip_log_dns_cate[ip_log_dns_cate['ip'].isin(most_mid_friends)]
most_domain_counts = most_dns_cate.domain_category.value_counts()
most_domain_counts = pd.DataFrame(most_domain_counts)
most_domain_counts.drop('unknown',inplace=True)

In [20]:
##可视化攻击者攻击域名行业的汇总统计
cate_pie = Pie("",width = 750, height=800)
cate_pie.add("域名类别",
    list(most_domain_counts.index),
    list(most_domain_counts.domain_category),
    is_label_show=True,
    label_pos='left'
    )

In [21]:
cate_pie.render('ip_file\\最具攻击性ip的攻击域名行业统计.html')

In [22]:
##从攻击者的流量日志中角度分析攻击者常用的网络攻击手法，以及攻击次数得分
logs_ip_score = pd.read_csv('ip_file\\logs_ip_score.csv',delimiter=',',encoding='utf-8')
most_ip_score = logs_ip_score[logs_ip_score['ip'].isin(most_mid_friends)]
most_ip_score_by_msg = most_ip_score.groupby('msg').sum()
most_ip_score_by_msg=most_ip_score_by_msg.sort_values(by='score')
most_ip_score_by_msg.drop('MSG',inplace=True)
most_ip_score_sum=most_ip_score_by_msg.score.sum()
most_ip_score_by_msg['percent'] = most_ip_score_by_msg['score'].apply(lambda x:x/most_ip_score_sum)

In [23]:
#可视化网络攻击手法统计结果
attack_type_pie = Pie("",width = 1400, height=1600)
attack_type_pie.add("攻击手法",
    list(most_ip_score_by_msg.index),
    list(most_ip_score_by_msg.score),
    is_label_show=True,
    label_pos='left'
    )

In [24]:
attack_type_pie.render('ip_file\\最具攻击性ip的攻击手法.html')